In [ ]:
from google.colab import drive
from transformers import pipeline
from PIL import Image
from transformers import AutoProcessor, AutoModelForVision2Seq
from tqdm import tqdm
from pathlib import Path
import os
import glob
import json
drive.mount("/content/drive/") # Don't change this.

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [ ]:
#!unzip /content/drive/MyDrive/ProjektMAI/data.zip -d /content/drive/MyDrive/ProjektMAI/data

Archive:  /content/drive/MyDrive/ProjektMAI/data.zip
replace /content/drive/MyDrive/ProjektMAI/data/data/ocr_data_openai.json? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [ ]:

def find_png_files(path):
  """
  Finds all PNG files in a given directory and its subdirectories.

  Args:
    # path: The directory to search.

  Returns:
    A list of paths to all PNG files found.
  """

  png_files = []
  for root, _, files in os.walk(path):
    for file in files:
      if file.endswith(".png"):
        png_files.append(os.path.join(root, file))

  return png_files

# Find all PNG files in the specified directory and its subdirectories.
png_files = find_png_files("/content/drive/MyDrive/ProjektMAI/data")

# Print the paths of the found PNG files.
images = [(file, Image.open(file)) for file in tqdm(png_files)]



  0%|          | 0/6405 [00:00<?, ?it/s]

  0%|          | 1/6405 [00:10<18:08:47, 10.20s/it]

  1%|          | 43/6405 [00:10<17:58,  5.90it/s]  

  1%|▏         | 94/6405 [00:10<06:43, 15.64it/s]

  2%|▏         | 156/6405 [00:10<03:15, 31.89it/s]

  3%|▎         | 210/6405 [00:10<02:02, 50.37it/s]

  4%|▍         | 261/6405 [00:10<01:23, 73.22it/s]

  5%|▍         | 312/6405 [00:10<00:59, 101.77it/s]

  6%|▌         | 372/6405 [00:10<00:41, 144.69it/s]

  7%|▋         | 425/6405 [00:11<00:32, 183.16it/s]

  7%|▋         | 478/6405 [00:11<00:25, 228.86it/s]

  8%|▊         | 538/6405 [00:11<00:20, 287.61it/s]

  9%|▉         | 592/6405 [00:11<00:17, 324.26it/s]

 10%|█         | 644/6405 [00:11<00:15, 362.23it/s]

 11%|█         | 696/6405 [00:11<00:14, 385.54it/s]

 12%|█▏        | 746/6405 [00:11<00:13, 411.72it/s]

 13%|█▎        | 805/6405 [00:11<00:12, 444.64it/s]

 14%|█▎        | 873/6405 [00:11<00:10, 505.70it/s]

 15%|█▍        | 929/6405 [00:12<00:10, 511.48it/s]

 15%|█▌

In [ ]:
model = AutoModelForVision2Seq.from_pretrained("microsoft/kosmos-2-patch14-224").to("cuda")
processor = AutoProcessor.from_pretrained("microsoft/kosmos-2-patch14-224")

prompt = "<grounding> Give a very detailled description of the image:"

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.

  0%|          | 0/641 [00:04<?, ?it/s]


In [ ]:
def describe_image(image):
  inputs = processor(text=[prompt]*len(image), images=image, return_tensors="pt").to("cuda")

  generated_ids = model.generate(
      pixel_values=inputs["pixel_values"],
      input_ids=inputs["input_ids"],
      attention_mask=inputs["attention_mask"],
      image_embeds=None,
      image_embeds_position_mask=inputs["image_embeds_position_mask"],
      use_cache=True,
      max_new_tokens=256,
  )

  generated_text = processor.batch_decode(generated_ids, skip_special_tokens=True)
  processed_text = [processor.post_process_generation(image)[0] for image in generated_text]

  return processed_text


In [ ]:
processed_images = dict()
file_path = "/content/drive/MyDrive/ProjektMAI/caption_kosmos.json"

for i in tqdm(range(0, len(images), 10)):
    paths = [image[0] for image in images[i:i+10]]
    imgs = [image[1] for image in images[i:i+10]]
    images_descriptions =  describe_image(imgs)
    for path, image in zip(paths, imgs):
        processed_images[path]  =  images_descriptions

    # Save the processed_images dictionary to a JSON file
    with open(file_path, "w") as f:
        json.dump(processed_images, f)

100%|██████████| 641/641 [47:43<00:00,  4.47s/it]


In [ ]:
!ls 

In [59]:
import json

# Define the file path
file_path = "../../../data/caption_kosmos.json"

# Read the JSON file into a dictionary
with open(file_path, 'r') as file:
    json_dict = json.load(file)

json_dict_cleaned = dict()
keys_idx = range(len(json_dict))
for i in keys_idx:
    json_dict_cleaned[list(json_dict.keys())[i].split("/")[-1]] = json_dict[list(json_dict.keys())[i]][i % 10].replace("Give a very detailled description of the image: ", "")


In [30]:
json_dict

{'/content/drive/MyDrive/ProjektMAI/data/data/HVVMemes/images/covid_memes_4692.png': 'Give a very detailled description of the image: A man and a woman are standing next to each other, and the man is holding a red envelope. The caption reads, "When you accidentally quote the customer the wrong price."',
 '/content/drive/MyDrive/ProjektMAI/data/data/HVVMemes/images/memes_928.png': 'Give a very detailled description of the image: The image features two men sitting in white chairs in a grassy field. They are both smiling, and the man on the left is wearing a tie.',
 '/content/drive/MyDrive/ProjektMAI/data/data/HVVMemes/images/covid_memes_537.png': 'Give a very detailled description of the image: The image features a group of men standing together on a billboard, with one of them wearing a hat.',
 '/content/drive/MyDrive/ProjektMAI/data/data/HVVMemes/images/covid_memes_5229.png': 'Give a very detailled description of the image: A man is standing in a doorway, wrapped in a blanket. He is lo

In [ ]:
datasets = ["dev_test", "dev", "train"]

for dataset in datasets:
    file_path =  f"../../../data/HVVMemes/annotations/{dataset}.jsonl"
    jsonObj = pd.read_json(path_or_buf=file_path, lines=True)
    jsonObj = jsonObj.T[~jsonObj.columns.str.contains("GPT-4o")].T

    jsonObj = jsonObj.merge(df, right_index=True, left_on="image", how="left")


    with open(file_path, "w") as f:
        f.write(jsonObj.to_json(orient='records', lines=True))

Output hidden; open in https://colab.research.google.com to view.